In [1]:
import pandas as pd
import numpy as np
import os
import re


In [ ]:
# Specify the path to your text file
file_path = '../../../EDW Queries/CDR-notes-nlinque/PatientIDs.txt'

# Read the file into a DataFrame
id_df = pd.read_csv(file_path, sep=' ', header=0)
id_df['MRN'] = id_df['MRN'].astype(str)
id_df['DOB'] = pd.to_datetime(id_df['DOB'])
id_df['VisitDate'] = pd.to_datetime(id_df['VisitDate'])
# Display the DataFrame to check
id_df


In [ ]:
visits_df = pd.read_csv("../../../../../NLP/Samad/MEM_data/df_files/2022_02_16_MDU_Latest_Visits.csv")
visits_df['CompleteMRN'] = visits_df['MRN'].str.split('-')
visits_df['MRN'] = visits_df['CompleteMRN'].str.join('')
visits_df['DOB'] = pd.to_datetime(visits_df['DOB'])
visits_df['VisitDate'] = pd.to_datetime(visits_df['VisitDate'])
visits_df

In [ ]:
visits_df = id_df.merge(visits_df, how="left", on=["MRN", "DOB", "VisitDate"])
visits_df

In [ ]:
print(visits_df['MRN'].nunique())

## Filtering Cases

In [ ]:
notes_df = pd.read_csv("../../../EDW Queries/CDR-notes-nlinque/mdu_notes_formatted_aug14.csv")
notes_df['ContactDTS'] = pd.to_datetime(notes_df['ContactDTS'])
notes_df

In [8]:
substrings = ["The note for this office visit has been dictated", "THIS NOTE IS IN PROCESS"]
pattern = '|'.join(substrings)
notes_df = notes_df[~notes_df['NoteTXT'].str.contains(pattern)] 


In [ ]:
print(notes_df['PatientID'].nunique())
print(visits_df.columns)

In [ ]:
merged_df = pd.merge(notes_df, visits_df, on='PatientID', how='inner')

# Calculate the absolute difference between ContactDTS and VisitDate
merged_df['DateDiff'] = (merged_df['ContactDTS'] - merged_df['VisitDate']).abs()
filtered_df = merged_df[merged_df['InpatientNoteTypeDSC'] == "Progress Notes"]
filtered_df = filtered_df[filtered_df['EncounterTypeDSC'].isin(["Appointment", "Office Visit", "Telemedicine", "Telemedicine - audio only"])]
filtered_df = filtered_df[filtered_df['ContactDTS'] == filtered_df['VisitDate']]
# substrings = ["dictated", "THIS NOTE IS IN PROCESS"]
# pattern = '|'.join(substrings)
# filtered_df = filtered_df[~filtered_df['NoteTXT'].str.contains(pattern)] 
print(filtered_df['PatientID'].nunique())

# Sort by PatientID and DateDiff to find the closest encounter
# closest_encounters = filtered_df.loc[filtered_df.groupby('PatientID')['DateDiff'].idxmin()]
# # closest_encounters = closest_encounters[closest_encounters['DateDiff']<14]
# # closest_encounters
# closest_encounters = closest_encounters[closest_encounters['DateDiff'] < pd.Timedelta(weeks=1)]
# print(closest_encounters['PatientID'].nunique())

# Drop the 'DateDiff' column if no longer needed
# closest_encounters = closest_encounters.drop(columns=['DateDiff'])

# closest_encounters.reset_index(inplace=True)
# # The resulting DataFrame contains the closest ContactDTS to the VisitDate for each patient
# closest_encounters

In [ ]:
# check if any problematic notes
filtered_df[filtered_df['NoteTXT'].str.contains("THIS NOTE IS IN PROCESS")]
filtered_df[filtered_df['NoteTXT'].str.contains(pattern)] 

In [ ]:
filtered_df['PatientID'].value_counts()

In [ ]:
# Assume 'PatientID' is the column name for patient IDs
patients_with_multiple_entries = filtered_df.groupby('PatientID').filter(lambda x: len(x) > 1)
# Display the result
patients_with_multiple_entries = patients_with_multiple_entries.drop_duplicates(subset=['PatientID'])
patient_ids = patients_with_multiple_entries['PatientID'].tolist()
patient_ids

In [17]:
patients_duplicates = filtered_df[filtered_df['PatientID'].isin(patient_ids)]
patients_duplicates.to_csv("/Users/yl509/Partners HealthCare Dropbox/Yu Leng/NLP/Yu Leng/GPT/cdr_patients_multi_notes.csv", index=False)

In [25]:
# Concatenate notes for each patient (and visit if necessary)
concatenated_df = filtered_df.groupby(['PatientID', 'PatientEncounterID'])['NoteTXT'].agg(' '.join).reset_index()
temp_df = filtered_df.copy()
temp_df = temp_df.drop(columns=['NoteTXT'])
concatenated_df = concatenated_df.merge(temp_df, on=['PatientID', 'PatientEncounterID'], how='left')
concatenated_df = concatenated_df.drop_duplicates(subset=['PatientID', 'PatientEncounterID'])
concatenated_df.reset_index(inplace=True)

In [ ]:
concatenated_df['ContactDTS'].sort_values()

In [ ]:
concatenated_df_id = concatenated_df.copy()
concatenated_df_id = concatenated_df_id[['PatientID', 'ContactDTS', 'VisitDate', 'GlobalCDR']]
concatenated_df_id.to_csv("../../../../GPT/cdr_id_strict_0827.csv", index=False)

In [ ]:
# closest_encounters['ContactDTS'].sort_values()

## Cleaning Notes

In [ ]:
def clean_text(text):
    # Remove specific unwanted characters
    text = text.replace('¬†', '')
    text = text.replace('ý', '')
    # Optionally, remove other common unwanted characters
    text = text.replace('\xa0', ' ')  # Replace non-breaking spaces with regular spaces
    return text

processed_df = concatenated_df.copy()
# Apply the cleaning function to the entire column
processed_df['NoteTXT'] = processed_df['NoteTXT'].apply(clean_text)

# Display the DataFrame to verify the changes
print(processed_df['NoteTXT'])

In [ ]:
def remove_and_save_cdr_sentences(text):
    # Regex pattern to identify sentences containing CDR information (case-insensitive)
    pattern = r'([^.]*\bCDR\b[^.]*\.)'
    # Find all sentences that match the pattern
    removed_text = re.findall(pattern, text, flags=re.IGNORECASE)
    print(removed_text)
    # Remove the matched sentences from the original text
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE).strip()
    
    # Join the removed sentences into a single string
    removed_text_str = ' '.join(removed_text).strip()
    return cleaned_text, removed_text_str

# Apply the function to the NoteTXT column and create two new columns
processed_df[['CleanedNoteTXT', 'RemovedCDRSentences']] = processed_df['NoteTXT'].apply(
    lambda x: pd.Series(remove_and_save_cdr_sentences(x))
)

In [ ]:
processed_df

In [ ]:
cdr_exists = processed_df['NoteTXT'].str.contains(r'\bCDR\b', case=False, regex=True)

# Print rows where "CDR" still exists (if any)
remaining_cdr_notes = processed_df[cdr_exists]
print("Rows where 'CDR' is still present:")
print(remaining_cdr_notes)

# Alternatively, count how many such rows exist
num_cdr_present = cdr_exists.sum()
print(f"Number of rows where 'CDR' is still present: {num_cdr_present}")


In [ ]:
cdr_exists = processed_df['CleanedNoteTXT'].str.contains(r'\bCDR\b', case=False, regex=True)

# Print rows where "CDR" still exists (if any)
remaining_cdr_notes = processed_df[cdr_exists]
print("Rows where 'CDR' is still present:")
print(remaining_cdr_notes)

# Alternatively, count how many such rows exist
num_cdr_present = cdr_exists.sum()
print(f"Number of rows where 'CDR' is still present: {num_cdr_present}")

In [ ]:
print(processed_df['NoteTXT'][3])


In [ ]:
processed_df[["RemovedCDRSentences", "GlobalCDR"]]

In [ ]:
# processed_df.to_csv("/Users/yl509/Partners HealthCare Dropbox/Yu Leng/NLP/Yu Leng/GPT/cdr_preprocessed_strict_0827.csv")
processed_df.to_csv("../../../../GPT/cdr_preprocessed_strict_0829.csv")

In [96]:
# processed_df.to_csv("/Users/yl509/Partners HealthCare Dropbox/Yu Leng/NLP/Yu Leng/GPT/cdr_preprocessed.csv")

In [ ]:
processed_df = pd.read_csv("../../../../GPT/cdr_preprocessed.csv")
processed_df

In [ ]:
processed_df['ContactDTS'].sort_values()